# Shoonya API Testing

In [2]:
# ! pip install ..\dist\NorenRestApi-0.0.30-py2.py3-none-any.whl

In [10]:
from NorenRestApiPy.NorenApi import  NorenApi
from threading import Timer
import pandas as pd
import time
import concurrent.futures

api = None
class Order:
     def __init__(self, buy_or_sell:str = None, product_type:str = None,
                 exchange: str = None, tradingsymbol:str =None, 
                 price_type: str = None, quantity: int = None, 
                 price: float = None,trigger_price:float = None, discloseqty: int = 0,
                 retention:str = 'DAY', remarks: str = "tag",
                 order_id:str = None):
        self.buy_or_sell=buy_or_sell
        self.product_type=product_type
        self.exchange=exchange
        self.tradingsymbol=tradingsymbol
        self.quantity=quantity
        self.discloseqty=discloseqty
        self.price_type=price_type
        self.price=price
        self.trigger_price=trigger_price
        self.retention=retention
        self.remarks=remarks
        self.order_id=None


    #print(ret)

    


def get_time(time_string):
    data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')

    return time.mktime(data)


class ShoonyaApiPy(NorenApi):
    def __init__(self):
        NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')        
        global api
        api = self

    def place_basket(self, orders):

        resp_err = 0
        resp_ok  = 0
        result   = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

            future_to_url = {executor.submit(self.place_order, order): order for order in  orders}
            for future in concurrent.futures.as_completed(future_to_url):
                url = future_to_url[future]
            try:
                result.append(future.result())
            except Exception as exc:
                print(exc)
                resp_err = resp_err + 1
            else:
                resp_ok = resp_ok + 1

        return result
                
    def placeOrder(self,order: Order):
        ret = NorenApi.place_order(self, buy_or_sell=order.buy_or_sell, product_type=order.product_type,
                            exchange=order.exchange, tradingsymbol=order.tradingsymbol, 
                            quantity=order.quantity, discloseqty=order.discloseqty, price_type=order.price_type, 
                            price=order.price, trigger_price=order.trigger_price,
                            retention=order.retention, remarks=order.remarks)
        #print(ret)

        return ret

In [13]:
from dotenv import load_dotenv
import os 
from pyotp.totp import TOTP
from pprint import pprint


load_dotenv()


api = ShoonyaApiPy()
ret = api.login(userid=os.environ['user'], password=os.environ['pwd'], twoFA=TOTP(os.environ['factor2']).now(), vendor_code=os.environ['vc'], api_secret=os.environ['api_key'], imei=os.environ['imei'])
pprint(ret)

{'access_type': ['WEB', 'TT', 'MOB', 'API'],
 'actid': 'FA133525',
 'brkname': 'FINV',
 'brnchid': 'HO',
 'dmsg': '',
 'email': 'mohitrajrathor@gmail.com',
 'exarr': ['NSE',
           'NIPO',
           'NFO',
           'CDS',
           'BSE',
           'BCD',
           'MCX',
           'NCX',
           'BSTAR',
           'BFO'],
 'exch_prd': [{'exch': 'NSE', 'prds': ['C', 'I', 'H', 'B']},
              {'exch': 'NIPO', 'prds': ['C']},
              {'exch': 'NFO', 'prds': ['M', 'I', 'H', 'B']},
              {'exch': 'CDS', 'prds': ['M', 'I', 'H', 'B']},
              {'exch': 'BSE', 'prds': ['C', 'I', 'H', 'B']},
              {'exch': 'BCD', 'prds': ['M', 'I']},
              {'exch': 'MCX', 'prds': ['M', 'I', 'H', 'B']},
              {'exch': 'NCX', 'prds': ['M', 'I']},
              {'exch': 'BSTAR', 'prds': ['C']},
              {'exch': 'BFO', 'prds': ['M', 'I']}],
 'lastaccesstime': '1749108022',
 'lastpwdtime': '1749107362',
 'm_num': '9462443361',
 'mws': {'1': [{'cn

In [17]:
api.get_holdings()

[]

In [19]:
import pandas as pd 

data = pd.DataFrame(api.searchscrip(exchange='NSE', searchtext="reliance")['values'])
data

,exch,token,tsym,cname,instname,symname,seg,pp,ls,ti
0,NSE,2885,RELIANCE-EQ,RELIANCE INDUSTRIES LTD,EQ,RELIANCE,EQT,2,1,0.10


In [20]:
api.get_quotes(exchange="NSE", token="2885")


{'request_time': '12:58:34 05-06-2025',
 'stat': 'Ok',
 'exch': 'NSE',
 'tsym': 'RELIANCE-EQ',
 'cname': 'RELIANCE INDUSTRIES LTD',
 'symname': 'RELIANCE',
 'seg': 'EQT',
 'instname': 'EQ',
 'isin': 'INE002A01018',
 'pp': '2',
 'ls': '1',
 'ti': '0.10',
 'mult': '1',
 'lut': '1749108513',
 'uc': '1565.80',
 'lc': '1281.20',
 'wk52_h': '1608.80',
 'wk52_l': '1114.84',
 'toi': '169071500',
 'issuecap': '13532372898.000000',
 'cutof_all': 'false',
 'prcftr_d': '(1 / 1 ) * (1 / 1)',
 'token': '2885',
 'lp': '1446.90',
 'c': '1423.50',
 'h': '1454.70',
 'l': '1423.60',
 'ap': '1442.08',
 'o': '1428.00',
 'v': '7288048',
 'ltq': '6',
 'ltt': '12:58:33',
 'ltd': '05-06-2025',
 'tbq': '461988',
 'tsq': '1021308',
 'bp1': '1446.70',
 'sp1': '1447.00',
 'bp2': '1446.60',
 'sp2': '1447.40',
 'bp3': '1446.50',
 'sp3': '1447.50',
 'bp4': '1446.40',
 'sp4': '1447.60',
 'bp5': '1446.30',
 'sp5': '1447.70',
 'bq1': '770',
 'sq1': '67',
 'bq2': '594',
 'sq2': '585',
 'bq3': '3537',
 'sq3': '175',
 'bq4